In [5]:
import numpy as np
import math as m
from scipy.constants import h, c, hbar, u, k
from scipy.optimize import curve_fit
from scipy.interpolate import CubicSpline
from ipynb.fs.full.Conversions_and_Constants import *

## [1]  $\mu$ (Mu)

In [6]:
#INPUT: AU

def mu(m_A, m_A_err, m_B, m_B_err):
    
    #SI
    mu_SI = ((m_A * m_B) / (m_A + m_B)) * u_to_kg
    #Error
    A = ((m_A_err * u_to_kg * m_B**2) / (m_A + m_B)**2)
    B = ((m_B_err * u_to_kg * m_A**2) / (m_A + m_B)**2)
    C = ((u_to_kg_err * m_A * m_B)  / (m_A + m_B))
    mu_SI_err = A + B + C
    #Other errors
    mu_SI_relerr = (mu_SI_err / mu_SI)
    mu_SI_pererr = mu_SI_relerr * 100
    
    #AU
    mu_au = mu_SI * kg_to_me
    mu_au_err = (kg_to_me * mu_SI_err) + (mu_SI * kg_to_me_err)
    #Other errors
    mu_au_relerr = mu_au_err/mu_au
    mu_au_pererr = mu_au_relerr * 100
    
    return mu_SI, mu_SI_err, mu_SI_relerr, mu_SI_pererr, mu_au, mu_au_err, mu_au_relerr, mu_au_pererr

#OUTPUT: SI - kg, AU - me

## [2] $\chi^2$, (Chi Squared)

In [1]:
def chisquare(observed_values,expected_values):
    test_statistic=0
    for observed, expected in zip(observed_values, expected_values):
        test_statistic+= abs((float(observed)-float(expected))**2/float(expected))
    return test_statistic

## [3]  Eta(II), $\eta_{II}$

In [3]:
#INPUT: AU

def eta_ii(zeta_ii, zeta_ii_err, mu_au, mu_au_err):
    
    eta_ii = zeta_ii/(mu_au)
    
    eta_ii_err = abs(zeta_ii_err/(mu_au) - ((zeta_ii * mu_au_err)/(mu_au**2)))
    eta_ii_relerr = eta_ii_err/eta_ii
    eta_ii_pererr = (eta_ii_relerr * 100)
    
    return eta_ii, eta_ii_err, eta_ii_pererr

#OUTPUT: AU

## [4] Eta(III), $\eta_{III}$

In [ ]:
#INPUT: AU

def eta_iii(zeta_iii, zeta_iii_err, mu_au, mu_au_err):
    
    eta_iii = zeta_iii/(mu_au**1.5)
    
    eta_iii_err = abs(zeta_iii_err/(mu_au**1.5) - ((3 * zeta_iii * mu_au_err)/(2 * mu_au**2.5)))
    eta_iii_relerr = eta_iii_err/eta_iii
    eta_iii_pererr = (eta_iii_relerr * 100)
    
    return eta_iii, eta_iii_err, eta_iii_pererr

#OUTPUT: AU

## [5] Eta(IIII), $\eta_{IIII}$

In [2]:
#INPUT: AU

def eta_iiii(zeta_iiii, zeta_iiii_err, mu_au, mu_au_err):
    
    eta_iiii = zeta_iiii/(mu_au**2)
    
    eta_iiii_err = abs(zeta_iiii_err/(mu_au**2) - ((2 * zeta_iiii * mu_au_err)/((mu_au**2))))
    eta_iiii_relerr = eta_iiii_err/eta_iiii
    eta_iiii_pererr = (eta_iiii_relerr * 100)
    
    return eta_iiii, eta_iiii_err, eta_iiii_pererr

#OUTPUT: AU

## [6] Omega, $\omega$ 

In [4]:
#INPUT: AU

def omega(eta_ii, eta_ii_err):
    
    #AU
    omega_au = np.sqrt(eta_ii)
    #Error
    omega_au_err = 1/(2*omega_au**0.5) * eta_ii_err
    omega_au_relerr = omega_au_err/omega_au
    omega_au_pererr = omega_au_relerr*100
    
    #SI
    omega_wn = omega_au * hartree_to_wn
    #Error
    omega_wn_err = (hartree_to_wn * omega_au_err) + (omega_au * hartree_to_wn_err)
    omega_wn_relerr = omega_wn_err/omega_wn
    omega_wn_pererr = omega_wn_relerr*100
    
    return omega_au, omega_au_err, omega_au_pererr, omega_wn, omega_wn_err, omega_wn_pererr

#OUTPUT: AU & SI

## [7] Sigma, $\sigma$

In [ ]:
#INPUT: AU

def sigma(eta_iii, eta_iii_err, omega_au, omega_au_err):
    
    sigma = -(5/12) * (eta_iii/omega_au**3)
    
    #Error
    sigma_err = abs(((-5*eta_iii_err)/(12*omega_au**3)) + ((15*eta_iii*omega_au_err)/(12*omega_au**4)))
    sigma_relerr = sigma_err/sigma
    sigma_pererr = sigma_relerr*100
    
    return sigma, sigma_err, sigma_pererr
    
#OUTPUT: AU

## [8]  $\Delta E^{(TOSH)}$ 

In [6]:
#INPUT: AU

def E_TOSH(omega_au, omega_au_err, eta_iii, eta_iii_err, eta_iiii, eta_iiii_err, sigma, sigma_err):

    #AU
    E_TOSH_au = omega_au + (eta_iiii/(8*omega_au**2)) + ((eta_iii * sigma)/(2*omega_au)) + ((eta_iiii*sigma**2)/(4*omega_au))
    #Error
    A = omega_au_err * (1 - (eta_iiii / (8 * omega_au**3)) - ((eta_iii * sigma)/(2 * omega_au**2)))
    B = eta_iiii_err * ((1 / (8 * omega_au**2)))
    C = eta_iii_err * (sigma / (2 * omega_au))
    D = sigma_err * ((eta_iii / (2 * omega_au)))
    E_TOSH_au_err = abs(A + B + C + D)
    E_TOSH_au_relerr = E_TOSH_au_err/E_TOSH_au
    E_TOSH_au_pererr = E_TOSH_au_relerr * 100
    
    #SI
    E_TOSH_wn = E_TOSH_au * hartree_to_wn
    #Error
    E_TOSH_wn_err = (E_TOSH_au * hartree_to_wn_err) + (hartree_to_wn * E_TOSH_au_err)
    E_TOSH_wn_relerr = E_TOSH_wn_err/E_TOSH_wn
    E_TOSH_wn_pererr = abs(E_TOSH_wn_relerr *100)
    
    return E_TOSH_au, E_TOSH_au_err, E_TOSH_au_pererr, E_TOSH_wn, E_TOSH_wn_err, E_TOSH_wn_pererr

#OUTPUT: AU & wn    


## [9] $\Delta E^{(VPT2)}$ 

In [7]:
#INPUT: AU

def E_VPT2(omega_au, omega_au_err, eta_iii, eta_iii_err, eta_iiii, eta_iiii_err, sigma, sigma_err):
    
    #AU
    E_VPT2_au = omega_au + (eta_iiii / (8 * omega_au**2)) - ((5 * eta_iii**2) / (24 * omega_au**4)) - ((eta_iiii**2) / (32 * omega_au**4))
    #Error
    A = omega_au_err * (1 - (eta_iiii / (4 * omega_au**3)) + ((5 * eta_iii**2) / (6 * omega_au**5)) + (eta_iiii**2 / (8*omega_au**5)))
    B = eta_iii_err * (-5 * eta_iii / (12 * omega_au**4))
    C = eta_iiii_err * ((1/(8 * omega_au**2)) - ((eta_iii) / (16 * omega_au**4)))
    E_VPT2_au_err = abs(A + B + C)
    E_VPT2_au_relerr = E_VPT2_au_err / E_VPT2_au
    E_VPT2_au_pererr = (E_VPT2_au_relerr * 100)
    
    #SI
    E_VPT2_wn = E_VPT2_au * hartree_to_wn
    #Error
    E_VPT2_wn_err = (E_VPT2_au * hartree_to_wn_err) + (hartree_to_wn * E_VPT2_au_err)
    E_VPT2_wn_relerr = E_VPT2_wn_err/E_VPT2_wn
    E_VPT2_wn_pererr = abs(E_VPT2_wn_relerr * 100)
    
    return E_VPT2_au, E_VPT2_au_err, E_VPT2_au_pererr, E_VPT2_wn, E_VPT2_wn_err, E_VPT2_wn_pererr
    
#OUTPUT: AU & SI

## [10] Prometheus Rotational Constant, $B_0$

In [9]:
#INPUT: AU

def B_0(mu_SI, mu_SI_err, re_au, re_au_err):
    
    #Bond length for v == 0
    r_0_au = re_au 
    r_0_au_err = (re_au_err)
    #SI
    r_0_SI = r_0_au * bohr_to_m
    r_0_SI_err = (bohr_to_m)*r_0_au_err
    
    #Rotational Constant v == 0
    B_0_wn = h / (8 * (np.pi**2) * c_cm * mu_SI * r_0_SI**2) 
    A = mu_SI_err * (-h / (8 * np.pi**2 * c_cm * mu_SI**2  * r_0_SI**2))
    B = r_0_SI_err * (-h / (8 * np.pi**2 * c_cm * mu_SI  * r_0_SI**3))
    B_0_wn_err = abs(A + B)
    B_0_wn_relerr = B_0_wn_err / B_0_wn
    B_0_wn_pererr = B_0_wn_relerr * 100
    
    return B_0_wn, B_0_wn_err, B_0_wn_pererr

#OUTPUT: wn

## [10] Prometheus Rotational Constant, $B_1$ 

In [8]:
#INPUT: AU

def B_1(mu_au, mu_au_err, mu_SI, mu_SI_err, re_au, re_au_err, sigma, sigma_err):
    
    #Bond length for v == 1
    r_1_au = (re_au + (sigma / mu_au**0.5))
    #Error
    A = re_au_err
    B = sigma_err * (1 / mu_au**0.5) 
    C = mu_au_err * (-sigma / (2 * mu_au**(3/2)))
    r_1_au_err = abs(A + B + C)
    #SI
    r_1_SI = r_1_au * bohr_to_m
    r_1_SI_err = (bohr_to_m)*r_1_au_err
    
    #Rotational Constant v == 1
    B_1_wn = h / (8 * (np.pi**2) * c_cm * mu_SI * r_1_SI**2) 
    A = mu_SI_err * (-h / (8 * np.pi**2 * c_cm * mu_SI**2  * r_1_SI**2))
    B = r_1_SI_err * (-h / (8 * np.pi**2 * c_cm * mu_SI  * r_1_SI**3))
    B_1_wn_err = abs(A + B)
    B_1_wn_relerr = B_1_wn_err / B_1_wn
    B_1_wn_pererr = B_1_wn_relerr * 100
    
    return B_1_wn, B_1_wn_err, B_1_wn_pererr

#OUTPUT: wn

## [11] Transition: $v_O$ 

In [12]:
def v_O_transition(J, omega, b0, b1):
    v_O = omega + b1*((J**2) + J) - b0*((J**2) + (5*J) + 6)
    return v_O

In [13]:
def v_O_errors(J, omega_0_err, b0_err, b1_err, v_O):
    abs_error = omega_0_err + (J**2 + J)*b1_err + (J**2 + (5*J) + 6)*b0_err
    rel_error = abs_error/v_O
    per_error = abs(rel_error) * 100
    return abs_error, rel_error, per_error

In [14]:
def v_O_errors_no_origin(J, b0_err, b1_err, v_O):
    abs_error = (J**2 + J)*b1_err + (J**2 + (5*J) + 6)*b0_err
    rel_error = abs_error/v_O
    per_error = abs(rel_error) * 100
    return abs_error, rel_error, per_error

## [12] Transition: $v_P$ 

In [15]:
def v_P_transition(J, omega, b0, b1):
    v_P = omega - ((b1 + b0)*(J+1)) + ((b1 - b0)*(J+1)**2)
    return v_P

In [16]:
def v_P_errors(J, omega_0_err, b0_err, b1_err, v_P):
    abs_error = omega_0_err + (-(J+1) + (J+1)**2)*b1_err + (-(J+1)-(J+1)**2)*b0_err
    rel_error = abs_error/v_P
    per_error = abs(rel_error) * 100
    return abs_error, rel_error, per_error

In [17]:
def v_P_errors_no_origin(J, b0_err, b1_err, v_P):
    abs_error = (-(J+1) + (J+1)**2)*b1_err + (-(J+1)-(J+1)**2)*b0_err
    rel_error = abs_error/v_P
    per_error = abs(rel_error) * 100
    return abs_error, rel_error, per_error

## [13] Transition: $v_Q$ 

In [18]:
def v_Q_transition(J, omega, b0, b1):
    v_Q = omega + (J*(J+1)*(b1 - b0))
    return v_Q

In [19]:
def v_Q_errors(J, omega_0_err, b0_err, b1_err, v_Q):
    abs_error = omega_0_err + (J**2 + J)*b1_err + (-J**2 - J)*b0_err
    rel_error = abs_error/v_Q
    per_error = abs(rel_error) * 100
    return abs_error, rel_error, per_error

In [20]:
def v_Q_errors_no_origin(J, b0_err, b1_err, v_Q):
    abs_error = (J**2 + J)*b1_err + (-J**2 - J)*b0_err
    rel_error = abs_error/v_Q
    per_error = abs(rel_error) * 100
    return abs_error, rel_error, per_error

## [14] Transition: $v_R$ 

In [21]:
def v_R_transition(J, omega, b0, b1):
    v_R = omega + ((b1 + b0)*(J+1)) + ((b1 - b0)*(J+1)**2)
    return v_R

In [22]:
def v_R_errors(J, omega_0_err, b0_err, b1_err, v_R):
    abs_error = omega_0_err + ((J+1) + (J+1)**2)*b1_err + ((J+1)-(J+1)**2)*b0_err
    rel_error = abs_error/v_R
    per_error = abs(rel_error) * 100
    return abs_error, rel_error, per_error

In [23]:
def v_R_errors_no_origin(J, b0_err, b1_err, v_R):
    abs_error = ((J+1) + (J+1)**2)*b1_err + ((J+1)-(J+1)**2)*b0_err
    rel_error = abs_error/v_R
    per_error = abs(rel_error) * 100
    return abs_error, rel_error, per_error

## [15] Transition: $v_S$ 

In [24]:
def v_S_transition(J, omega, b0, b1):
    v_S = omega + b1*((J**2) + (5*J) + 6) - b0*((J**2) + J)
    return v_S

In [25]:
def v_S_errors(J, omega_0_err, b0_err, b1_err, v_S):
    abs_error = omega_0_err - (J**2 + J)*b0_err + (J**2 + (5*J) + 6)*b1_err
    rel_error = abs_error/v_S
    per_error = abs(rel_error) * 100
    return abs_error, rel_error, per_error

In [26]:
def v_S_errors_no_origin(J, b0_err, b1_err, v_S):
    abs_error = - (J**2 + J)*b0_err + (J**2 + (5*J) + 6)*b1_err
    rel_error = abs_error/v_S
    per_error = abs(rel_error) * 100
    return abs_error, rel_error, per_error